In [1]:
import os
os.chdir('../')
%pwd

'/Users/pranavsrinivasvenkatesh/Projects/Data Science Projects/Kindey DIsease Classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [6]:
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()


In [7]:
class Training:
    def __init__(self, config: TrainingConfig):
        # MUST be the first thing done → fixes pyfunc registry errors
        #tf.config.run_functions_eagerly(True)
        #tf.data.experimental.enable_debug_mode()

        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False
        )
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [8]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-11-23 23:55:51,534: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-11-23 23:55:51,536: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-23 23:55:51,537: INFO: common: created directory at: artifacts]
[2025-11-23 23:55:51,538: INFO: common: created directory at: artifacts/training]


2025-11-23 23:55:51.553237: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-11-23 23:55:51.553262: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-11-23 23:55:51.553267: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-11-23 23:55:51.553282: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-23 23:55:51.553293: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.


/opt/miniconda3/envs/kidney/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 1/23 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.6875 - loss: 0.5908

2025-11-23 23:55:53.465375: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_10 in the registry.
Traceback (most recent call last):

  File "/opt/miniconda3/envs/kidney/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 258, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_10 in the registry.


2025-11-23 23:55:53.465889: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_10 in the registry.
Traceback (most recent call last):

  File "/opt/miniconda3/envs/kidney/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 258, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_10 in the registry.


	 [[{{

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ValueError: Could not find callback with key=pyfunc_10 in the registry.
Traceback (most recent call last):

  File "/opt/miniconda3/envs/kidney/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 258, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_10 in the registry.


	 [[{{node EagerPyFunc}}]] [Op:IteratorGetNext] name: 